# Inf2 - Foundations of Data Science
## Week 09: K-Means

**Learning outcomes:** 
In this lab you will implement the K-Means algorithm from scratch.

By the end of the lab you should be able to:
- implement and explain the different steps involved in K-Means,
- explain what the benefits of clustering algorithms are,
- explain what partitional clustering algorithms are,
- and be able to use sklearn's K-Means algorithm.

**Research question:**

In this lab we will use K-means on the same breast cancer patients data to see how many types of cancer can be distinguished.

**Data set information:** 

The data set is adapted from [UCI](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). It contains patient information taken from patients with breast cancer. 

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
#Importing sklearn functions
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import KMeans

Clustering refers to a group of unsupervised learning algorithms, one of which is K-means. Each cluster is a group of data points in which points should be more similar to each other than they are to points in other clusters. Clustering is used to interpret and compress data. Interpretation can help to deepen our knowledge of the dataset. For example in medicine, we might be interested in particular clusters of symptoms of a disease. For example, researchers at King's College have identified six clusters of patterns of [Coronavirus symptoms over time](https://www.medrxiv.org/content/10.1101/2020.06.12.20129056v1.full.pdf). The cluster labels can then be used as independent variables with hospitalisation as dependent variable to build a model predicting the likelihood of hospitalisation given a cluster membership.

K-Means belongs to the group of *partitional clustering* algorithms, in which we partition the data into $K$ non-overlapping clusters; each data point can only belong to exactly one cluster. If you are interested in other clustering algorithms, such as *hierarchical* or *density based* clustering, we invite you to read the following [paper](https://link.springer.com/content/pdf/10.1007/s40745-015-0040-1.pdf), or take IAML in your honours years.

Partitional clustering algorithms are non-deterministic, which means that each run can lead to different clusters even on the same data set. The main benefits of partitional clustering algorithms are their simplicity and low computational complexity. However, they are also only well suited for spherical clusters and do not cope well with complex shapes. 

The following pseudo code should help you remember from the lecture how the algorithm works:  

In [ ]:
# Choose k centroids randomly
# While(centroids are not the same as in the previous iteration) do:
    # Assign each data point to its closest centroid
    # Compute new centroid based on the mean of all the points in the cluster
# end while

## 9.A Implementing K-Means

The first step is to initialize the centroids. The easiest way would be to just take the first `K` data points. However, we also know that the result of K-Means depends on the initial centroids, and thus a random assignment is beneficial to allow for more restarts.

**Exercise 01:** 
- Create a function `initialize_centroids()`. It should take as parameters `data`, a numpy 2-dimensional array in which each row is an instance ($\mathbf x_i$ in the lecture notes) and `n_clusters` the number of clusters we are looking for ($K$ in the lecture notes). (Remark: we use `n_clusters` instead of `k_clusters`, as this is the notation used in sklearn and it won't confuse you once you use the sklearn functions.)
- Create an array `random_data_points` with `n_clusters` random integers in the range from 0 to the length of the data set. 

Hint: You can either create a loop which gets a [random number](https://pynative.com/python-random-randrange/) between 0 and the length of the data set, and appends the value to the `random_data_points` array (remember that each value appended to the array needs to be distinct), or you can use [numpy's](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.permutation.html) `random.permutation` function to create an array of same length as the data set with randomly permutated integers between 0 and length of the data set, and take the first `n_clusters` values of that array and assign it to `random_data_points`. 

- Finally, create a new numpy array by filtering from the data sets the rows corresponding to the randomly chosen numbers. and return it.

In [ ]:
# Your code


The next step consists of assigning each data point to a cluster.

**Exercise 02:**
- Create a function `assign_clusters()`. It should take as parameters `data` and `centroids`.
- Use `pairwise_distance(data, centroids, metric='euclidean')` from the sklearn library (which we have imported above) to compute the distance of each data point to each of the centroids. Look up the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html) to understand how the function works, and what the return value is. Store the returned value in a new variable `euclidean_distance`.
- Find the centroid that is closest to each data point, and store the array with the numbers representing the centroids for each data point in an array `assigned_clusters`. Hint: numpy's `argmin()` might help to find the centroid with the minimal distance. 
- Return `assigned_clusters`.

In [ ]:
# Your code


The last step to compute the optimal centroids is to update the centroids, based on their assignment of the previous iteration.

**Exercise 03:**
- Create a function `update_centroids()`. It should take as parameters `data`, `n_clusters` and `assigned_clusters`.
- For each cluster, compute the mean for each feature of all the data points in the cluster. You can use a loop or a list comprehension, and remember the numpy mean() function - but make sure you get the `axis` argument right.
- Create a 2-dimensional array `centroids` such that each row in the matrix contains the list of means computed for each cluster.
- Return the `centroids` as a 2-dimensional array, with each row containing one centroid.

Hint: The type of the final returned value should be a ndarray.

In [ ]:
# Your code


Now we have all the auxiliary functions that we need to finish to write our K-means algorithm. We are going to structure the code in a similar way to sklearn.

**Remark:** For every model sklearn implements there is a class with  `fit()` and `predict()` methods. The `fit()` method trains the model and the `predict()` method allows us to determine the classification of a seen or unseen data point. The term "predict" is perhaps a bit misleading in the case of K-means, where `.predict()` means "determine the cluster of a data point".

**Exercise 04:**
We have prepared a skeleton for a class called `K_Means`. We have also already prepared the constructor. Your job is now to add two methods: `fit()` and `predict()`.
- For the `fit()` method, look at the pseudo code above, and use the three functions you have coded previously to fit the model. Store the final centroids to `self.centroids`.
- The `predict()` method should take only one observation as input and compute to which centroid the distance is minimal, and then return the value of the centroid. (Careful: It will probably return an array with just one entry, but it should just return an integer.) 

In [ ]:
class K_Means:
    def __init__(self, n_clusters=2):
        self.n_clusters=n_clusters
# Your code        


**Discussion:** Before we apply the K-means algorithm, it is a good idea to standardize your data. Try to remember with your lab partner why standardization is important.

Your answer:

## 9.B Applying K-Means

We are given a data set of breast cancer patients. Our aim is to see if we can cluster the breast cancer data into two categories.

**Exercise 05:** 
- Load the `breast_cancer.csv` data set from `datasets`. We have preprocessed the data set for you. All unnecessary columns and rows with NaN values have been removed.
- We need to standardize the `breast_cancer` data. We can either
  - use the standardize() from the last lab or 
  - use the sklearn `StandardScaler()` class. Here is the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).
- Display your standardized data sets.

In [ ]:
# Your code


Let us now run our K-means algorithm on our data set.

**Exercise 06:**
- Instantiate a `K_Means` object with `n_clusters=2` parameters.
- Fit the model with your standardized data set.
- Print the centroids.
- Use [sklearn's native K-Means class](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) to compute the centroids, and print them out. Do you get similar results?

In [ ]:
# Your code


**Remark:** If you run the code several times, you might get a slightly different result every time.

**Exercise 07:**

In some cases, if the data set is very large the optimization can take a very long time. Add an object variable to your object called `max_iter` in the object constructor with a default value 100. Add an extra iteration variable to your loop and a condition to the loop that breaks the loop once the iterator is greater than `max_iter`.

In [ ]:
# Your code


**Exercise 08:**

Finally, we would like to plot the centroids. However, our data is high-dimensional. By now you should think "Too many dimensions? -> PCA".

- Apply PCA to your standardized dataset - we suggest you use 3 components, but you may wish to look at how much variance is explained by each component using `pca.explained_variance_ratio_`
- Apply K-means to the PCA scores, using `max_iter = 5`. (Run it several times to see whether the centroids remain the same.)
- Plot the scatter plot of the first two dimensions, colouring the two clusters differently.
- Plot the two centroids with different markers.
- Repeat the exercise with `max_iter=100`. (Run it several times to see, whether this time the centroids have converged.) 

In [ ]:
# Your code


In [ ]:
# Your code (for max_iter=100)


It looks like there are two reasonably well-separated clusters. Let's compare how well they match the benign and malignant clusters from the dataset from our previous lab.

**Exercise 09:**

- Load the `breast_cancer_diagnosis.csv` dataset, which in addition to the previous dimensions, contains the diagnosis `M` (malignant) or `B` (benign).
- Plot the scatter plot of the first two dimensions of the pca scores from the last exercise, but this time colouring the benign and malignant classes differently.

In [ ]:
# Your code


**Exercise 10:**
- Count the numbers of benign and malignant patients in each cluster.
- Which cluster best corresponds to each diagnosis?

In [ ]:
# Your code

**Discussion:**

You have learned a lot in the last few labs:

- Cleaning your data.
- Plotting your data.
- Applying PCA and K-Means.

And you have used all those skills in this lab. However, there are a couple of things you should think about. Discuss the following points with your lab partner.

- How could you prevent your model from overfitting to the data?
- We have already discussed `max_iter` as one option to stop the optimization if it takes too long. What is a different criterion based on which we could stop the optimization?

*Hint:* Look at the sklearn documentation of K-Means, the parameters might give you some ideas.

Your answer:

**We need your help:** This is a new course. In order for us to improve the labs for the next iterations, and to make sure that the next labs are better, we need your feedback. Please fill out the following [form](https://forms.office.com/Pages/ResponsePage.aspx?id=sAafLmkWiUWHiRCgaTTcYZmGMCx4KxlMjSTITqjdcXpUMTNCSlU4SzhaTVAyMjJGVlRWSUtaM1ZLQi4u).